In [1]:
import pandas as pd
import numpy as np
from datetime import date
import time

In [2]:
from bs4 import BeautifulSoup

In [3]:
from urllib.request import urlopen,Request

In [4]:
finviz_url = 'https://finviz.com/news.ashx'

In [5]:
req = Request(url=finviz_url , headers = {'user-agent':'my-app'})
response = urlopen(req)

In [6]:
html = BeautifulSoup(response,'html')
latest_news = html.find(id='news')

In [7]:
content = latest_news.find('table').findAll('table')[2]

In [8]:
data = []
today = date.today()
date = today.strftime('%m-%d-%Y')

for i in range(1,len(content)):
    if i%2==1:
        time = content.contents[i].text[2:9]
        title = content.contents[i].text[10:-1]
        data.append([date,time,title])
    else:
        'no'

In [9]:
news1 = pd.DataFrame(data,columns=['date','time','title'])

In [10]:
news = pd.read_csv('news.csv')

In [11]:
news = news.append(news1)[['date','time','title']]

In [12]:
news.to_csv('news.csv')

In [13]:
#To generate positive and negative reviews from news

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [14]:
vader = SentimentIntensityAnalyzer()

In [15]:
news['compound']=news['title'].apply(lambda x: vader.polarity_scores(x)['compound'])

In [16]:
review = []

for row in news['compound']: 
    if row<0:
        review.append('negative')
    elif row==0:
        review.append('neutral')
    else:  
        review.append('positive')

In [17]:
news['review']=review

In [18]:
#To extract organization name from news headlines

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [19]:
news['modified_title']=news['title'].apply(lambda x:x.replace('-','- '))

In [20]:
def org(x):
    doc = nlp(x)
    return [X.text for X in doc.ents if X.label_ == 'ORG']

In [21]:
news['organization']=news['modified_title'].apply(lambda x : org(x))

In [22]:
news.reset_index(inplace=True)

In [23]:
news=news.drop(columns='index',axis=1)

In [24]:
companies_list = pd.read_csv('companies_list.csv')

In [25]:
title=[]
search=[]
sector=[]
for j in range(0,len(news)):
    for i in range(0,len(companies_list)):
        if news['title'][j].lower().find(companies_list['Name'][i].lower())>=0:
            search1=companies_list['Name'][i]
            search.append(search1)
            title1=news['title'][j]
            title.append(title1)
            sector1=companies_list['Sector'][i]
            sector.append(sector1)
        else:
            search1='not found'

In [26]:
company_name_dataframe=pd.DataFrame({'title':title,'company_name':search,'sector':sector})

In [27]:
news_table=pd.merge(news,company_name_dataframe,how='inner')

In [28]:
news_table=news_table.drop(columns=['modified_title','organization'],axis=1)

In [29]:
import datetime

today = datetime.date.today()
last_monday = today - datetime.timedelta(days=today.weekday())-datetime.timedelta(days=7)
last_friday = today - datetime.timedelta(days=today.weekday())-datetime.timedelta(days=3)

In [30]:
table=news_table[(news_table['date']>=last_monday.strftime('%m/%d/%Y')) & (news_table['date']<=last_friday.strftime('%m/%d/%Y'))]

In [31]:
modified_news=table.groupby(['company_name','sector']).mean('compound')

In [32]:
review = []

for row in modified_news['compound']: 
    if row<0:
        review.append('negative')
    elif row==0:
        review.append('neutral')
    else:  
        review.append('positive')

In [33]:
modified_news['review']=review

In [34]:
positive=modified_news[modified_news['review']=='positive'].sort_values('compound',ascending=False).head(10)

In [35]:
negative=modified_news[modified_news['review']=='negative'].sort_values('compound').head(10)

In [36]:
last_friday

datetime.date(2021, 6, 18)

In [43]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys


recipients = ['hanisha.namburi@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = "companies having positive news"
msg['From'] = 'hanisha.namburi@gmail.com'


html = """\
<html>
  <head></head>
  <body>
    {0}
  </body>
</html>
""".format(positive.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)

try:
    """Checking for connection errors"""

    server = smtplib.SMTP('smtp.gmail.com', 587)
    #server.ehlo()#NOT NECESSARY
    server.starttls()
    #server.ehlo()#NOT NECESSARY
    server.login('hanisha.namburi@gmail.com','Hani@1995')
    server.sendmail(msg['From'], emaillist , msg.as_string())
    server.close()

except Exception as e:
    print("Error for connection: {}".format(e))


In [44]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys


recipients = ['hanisha.namburi@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = "companies having negative news"
msg['From'] = 'hanisha.namburi@gmail.com'


html = """\
<html>
  <head></head>
  <body>
    {0}
  </body>
</html>
""".format(negative.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)

try:
    """Checking for connection errors"""

    server = smtplib.SMTP('smtp.gmail.com', 587)
    #server.ehlo()#NOT NECESSARY
    server.starttls()
    #server.ehlo()#NOT NECESSARY
    server.login('hanisha.namburi@gmail.com','Hani@1995')
    server.sendmail(msg['From'], emaillist , msg.as_string())
    server.close()

except Exception as e:
    print("Error for connection: {}".format(e))